# Live visualization on a map

With [stackstac.show](https://stackstac.readthedocs.io/en/stable/api/main/stackstac.show.html) or [stackstac.add_to_map](https://stackstac.readthedocs.io/en/stable/api/main/stackstac.add_to_map.html), you can display your data on an interactive [ipyleaflet](https://ipyleaflet.readthedocs.io/en/latest/) map within your notebook. As you pan and zoom, the portion of the dask array that's in view is computed on the fly.

By running a Dask cluster colocated with the data, you can quickly aggregate hundreds of gigabytes of imagery on the backend, then only send a few megabytes of pixels back to your browser. This gives you a very simplified version of the [Google Earth Engine](https://developers.google.com/earth-engine/guides/playground) development experience, but with much more flexibility.

## Limitations

This functionality is still very much proof-of-concept, and there's a lot to be improved in the future:

1. Doesn't work well on large arrays. Sadly, loading a giant global DataArray and using the map to just view small parts of it won't work—yet. Plan on only using an array of the area you actually want to look at (passing `bounds_latlon=` to `stackstac.stack` helps with this).
2. Resolution doesn't change as you zoom in or out.
3. Communication to Dask can be slow, or seem to hang temporarily.
4. Requires opening port 8000 (only accessible over localhost), which may be blocked in some restrictive environments.

In [1]:
import stackstac
import pystac_client
import ipyleaflet
import xarray as xr
import IPython.display as dsp

In [2]:
import coiled
import distributed

You _definitely_ will need a cluster near the data for this (or to run on a beefy VM in `us-west-2`).

You can sign up for a Coiled account and run clusters for free at https://cloud.coiled.io/ — no credit card or username required, just sign in with your GitHub or Google account.

In [3]:
cluster = coiled.Cluster(
    name="stackstac-show",
    software="gjoseph92/stackstac",
    n_workers=22,
    worker_cpu=4,
    worker_memory="16GiB",
    scheduler_cpu=2,
    backend_options={"region": "us-west-2"},
)
client = distributed.Client(cluster)
client

Output()

Using existing cluster: 'stackstac-show'

<Client: 'tls://10.6.9.201:8786' processes=22 threads=88, memory=334.21 GiB>

Search for Sentinel-2 data overlapping our map

In [4]:
m = ipyleaflet.Map()
m.center = 35.677153763176115, -105.8485489524901
m.zoom = 10
m.layout.height = "700px"
m

Map(center=[35.677153763176115, -105.8485489524901], controls=(ZoomControl(options=['position', 'zoom_in_text'…

In [5]:
%%time
bbox=[m.west, m.south, m.east, m.north]
stac_items = pystac_client.Client.open(
    "https://earth-search.aws.element84.com/v0"
).search(
    bbox=bbox,
    collections=["sentinel-s2-l2a-cogs"],
    datetime="2020-04-01/2020-04-15"
).get_all_items()
len(stac_items)

CPU times: user 27.6 ms, sys: 5.19 ms, total: 32.8 ms
Wall time: 861 ms


24

In [6]:
dsp.GeoJSON(stac_items.to_dict())

<IPython.display.GeoJSON object>

## Create the time stack

**Important: the resolution you pick here is what the map will use, regardless of zoom level!** When you zoom in/out on the map, the data _won't_ be loaded at lower or higher resolutions. (In the future, we hope to support this.)

Beware of zooming out on high-resolution data; you could trigger a massive amount of compute!

In [7]:
%time stack = stackstac.stack(stac_items, resolution=80)

CPU times: user 18.2 ms, sys: 1.36 ms, total: 19.6 ms
Wall time: 18.8 ms


## Persist the data we want to view

By [persisting](https://docs.dask.org/en/latest/user-interfaces.html#persist-into-distributed-memory) all the RGB data, Dask will pre-load it and store it in memory, ready to use. That way, we can tweak what we show on the map (different composite operations, scaling, etc.) without having to re-fetch the original data every time. It also means tiles will load much faster as we pan around, since they're already mostly computed.

It's generally a good idea to persist somewhere before `stackstac.show`. Typically you'd do this after a reduction step (like a temporal composite), but our data here is small, so it doesn't matter much.

As a rule of thumb, try to persist after the biggest, slowest steps of your analysis, but before the steps you might want to tweak (like thresholds, scaling, etc.). If you want to tweak your big slow steps, well... be prepared to wait (and maybe don't persist).

In [8]:
client.wait_for_workers(22)

In [9]:
rgb = stack.sel(band=["B04", "B03", "B02"]).persist()
rgb

,Array,Chunk
Bytes,3.69 GiB,8.00 MiB
Shape,"(24, 3, 2624, 2622)","(1, 1, 1024, 1024)"
Count,648 Tasks,648 Chunks
Type,float64,numpy.ndarray


## `stackstac.add_to_map`

`stackstac.add_to_map` displays a DataArray on an existing ipyleaflet map. You give it a layer name—if a layer with this name already exists, it's replaced; otherwise, it's added. This is nice for working in notebooks, since you can re-run an `add_to_map` cell to adjust it, without piling up new layers.

Before continuing, you should open the distributed dashboard in another window (or use the [dask-jupyterlab](https://github.com/dask/dask-labextension) extension) in order to watch its progress.

In [10]:
m.zoom = 10
m

Map(bottom=103580.0, center=[35.677153763176115, -105.8485489524901], controls=(ZoomControl(options=['position…

`stackstac.server_stats` is a widget showing some under-the-hood stats about the computations currently running to generate your tiles. It shows "work bars"—like the inverse of progress bars—indicating the tasks it's currently waiting on.

In [11]:
stackstac.server_stats

VBox()

Make a temporal median composite, and show that on the map `m` above! Pan around and notice how the dask dashboard shows your progress.

In [12]:
comp = rgb.median("time")
stackstac.add_to_map(comp, m, "s2", range=[0, 3000])

TileLayer(name='s2', options=['attribution', 'detect_retina', 'max_native_zoom', 'max_zoom', 'min_native_zoom'…

Try changing `median` to `mean`, `min`, `max`, etc. in the cell above, and re-run. The map will update with the new layer contents (since you reused the name `"s2"`).

## Showing computed values

You can display anything you can compute with dask and xarray, not just raw data. Here, we'll compute NDVI (Normalized Difference Vegetation Index), which indicates the health of vegetation (and is kind of a "hello world" example for remote sensing).

In [13]:
nir, red = stack.sel(band="B08"), stack.sel(band="B04")
ndvi = (nir - red) / (nir + red)
ndvi = ndvi.persist()

We'll show the temporal maximum NDVI (try changing to `min`, `median`, etc.)

In [14]:
ndvi_comp = ndvi.max("time")

## `stackstac.show`

`stackstac.show` creates a new map for you, centers it on your array, and displays it. It's very convenient.

In [15]:
stackstac.show(ndvi_comp, range=(0, 0.6), cmap="YlGn")

Map(center=[36.090677085023124, -106.06931534400297], controls=(ZoomControl(options=['position', 'zoom_in_text…

To demonstrate more derived quantities: show each pixel's deviation from the mean NDVI of the whole array:

In [16]:
anomaly = ndvi_comp - ndvi.mean()

In [17]:
stackstac.show(anomaly, cmap="RdYlGn")

/Users/gabe/dev/stackstac/stackstac/show.py:484: UserWarning: Calculating 2nd and 98th percentile of the entire array, since no range was given. This could be expensive!
  warnings.warn(


Map(center=[36.090677085023124, -106.06931534400297], controls=(ZoomControl(options=['position', 'zoom_in_text…

## Interactively explore data with widgets

Using [ipywidgets.interact](https://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html#), you can interactively threshold the NDVI values by adjusting a slider. It's a bit clunky, and pretty slow to refresh, but still a nice demonstration of the powerful tools that become available by integrating with the Python ecosystem.

In [18]:
import ipywidgets

ndvi_map = ipyleaflet.Map()
ndvi_map.center = m.center
ndvi_map.zoom = m.zoom

@ipywidgets.interact(threshold=(0.0, 1.0, 0.1))
def explore_ndvi(threshold=0.2):
    high_ndvi = ndvi_comp.where(ndvi_comp > threshold)
    stackstac.add_to_map(high_ndvi, ndvi_map, "ndvi", range=[0, 1], cmap="YlGn")
    return ndvi_map

interactive(children=(FloatSlider(value=0.2, description='threshold', max=1.0), Output()), _dom_classes=('widg…